In [1]:
import numpy as np
from time import sleep

In [2]:
%%capture
!pip3 install robot_descriptions
from robot_descriptions.skydio_x2_mj_description import MJCF_PATH
from robot_descriptions.skydio_x2_description import URDF_PATH

In [3]:
from mujoco_simulator.model_builder import ModelBuilder
from mujoco_simulator.simulator import MujocoRender
model_wrapper = ModelBuilder(MJCF_PATH)
model_wrapper.add_arena()
model_wrapper.build()
ren = MujocoRender(model=model_wrapper, 
                   start=True, 
                   convention='pin')
Q_INIT = np.array([0, 0, 1.0, 0, 0, 0,1])
ren.set_state(Q_INIT)

Process render is starting!


In [4]:
from darli.model import Functional
from darli.backend import CasadiBackend, JointType
from darli.backend.liecasadi.so3 import SO3
from darli.state_space import StateSpace
import casadi as cs

model = Functional(CasadiBackend(URDF_PATH))
# TODO: add selector
selector = np.array([[0.0, 0.0, 0.0, 0.0],
                     [0.0, 0.0, 0.0, 0.0],
                     [1.0, 1.0, 1.0, 1.0],
                     [-0.18, 0.18, 0.18, -0.18],
                     [0.14, 0.14, -0.14, -0.14],
                     [-0.0201, 0.0201, 0.0201, -0.0201]])

model.update_selector(selector)

Define the variables in tangent space and get the configuration via exponential:

In [5]:

ns = 40

opti_problem = cs.Opti()
tf = opti_problem.parameter(1)
dt = tf / ns

state = opti_problem.variable(model.nq + model.nv, ns+1)
q = state[:model.nq, :]
v = state[model.nq:, :]
u = opti_problem.variable(model.nu, ns)

q0 = opti_problem.parameter(model.nq)
qf = opti_problem.parameter(model.nq)

In [6]:

position_weight = opti_problem.parameter(1)
# orientation_weight = opti_problem.parameter(1)
acceleration_weight = opti_problem.parameter(1)
velocity_weight = opti_problem.parameter(1)
control_weight = opti_problem.parameter(1)
control_max = opti_problem.parameter(model.nu)

fd = model.forward_dynamics

cost = 0 
for i in range(ns):
    
    dv_i = fd(q[:,i], v[:,i], u[:, i])
    opti_problem.subject_to(v[:, i+1] == v[:,i] + dt*dv_i)
    opti_problem.subject_to(q[:,i+1] == model.backend.integrate_configuration(q[:,i], v[:,i]*dt))
    
    cost += position_weight*cs.sumsqr(q[:3,i] - qf[:3])*dt
    # so3_error = SO3(xyzw=q[3:,i]).distance(SO3(xyzw = qf[3:]))**2
    # cost += orientation_weight*so3_error*dt
    cost += control_weight*cs.sumsqr(u[:,i])*dt
    cost += velocity_weight*cs.sumsqr(v[:,i])*dt
    # cost += acceleration_weight*cs.sumsqr(dv_i)*dt
    opti_problem.subject_to(opti_problem.bounded(np.zeros(model.nu), u[:, i], control_max))
    opti_problem.subject_to(q[2, i] >= 0.4)
    
opti_problem.subject_to(v[:,0] == np.zeros(model.nv))
opti_problem.subject_to(v[:, -1] == np.zeros(model.nv))
opti_problem.subject_to(q[:, 0] == q0)
opti_problem.subject_to(q[:, -1] == qf)

opti_problem.minimize(cost)
opti_problem.solver('ipopt')

In [7]:
np.random.seed(42)

In [8]:
Q_INIT = np.array([0, 0, 1.5, 0,0,0, 1])
quat_random = np.random.rand(4)
quat_random /= np.linalg.norm(quat_random)
Q_INIT[3:] = quat_random
Q_FINAL = np.array([0, 0., 2.5, 0, 0, 0, 1])
Q_FINAL[:3] += np.random.randn(3)

t_final = 3


opti_problem.set_value(control_max, 50*np.ones(model.nu) )
opti_problem.set_value(q0, Q_INIT)
opti_problem.set_value(qf, Q_FINAL)
opti_problem.set_value(tf[0], t_final)

opti_problem.set_value(position_weight[0], 5)
# opti_problem.set_value(orientation_weight[0], 0)
opti_problem.set_value(velocity_weight[0], 10)
opti_problem.set_value(acceleration_weight[0], 5)
opti_problem.set_value(control_weight[0], 5)
opti_problem.set_initial(q[3:,:], np.array([Q_INIT[3:]]*(ns+1)).T)
opti_problem.set_initial(q[:3,:], np.linspace(Q_INIT[:3], Q_FINAL[:3], ns+1).T)

ren.set_state(Q_INIT)
ren.markers[ns+1](position=Q_FINAL[:3],
                color=[1, 0, 0, 0.5],
                size=0.04)
ren.markers[ns+2](position=Q_INIT[:3],
                color=[0, 1, 0, 0.5],
                size=0.04)


In [9]:
try:
    sol = opti_problem.solve()
    q_opt = opti_problem.value(q)
    v_opt = opti_problem.value(v)
    u_opt = opti_problem.value(u)
except Exception as e:
    q_opt = opti_problem.debug.value(q)
    v_opt = opti_problem.debug.value(v)
    u_opt = opti_problem.debug.value(u)
    print(e)

CasADi - 2024-06-13 16:13:35 WARNING(".../casadi/core/casadi_os.cpp:166: Assertion "handle!=nullptr" failed:
PluginInterface::load_plugin: Cannot load shared library 'libcasadi_nlpsol_ipopt.so': 
   (
    Searched directories: 1. casadipath from GlobalOptions
                          2. CASADIPATH env var
                          3. PATH env var (Windows)
                          4. LD_LIBRARY_PATH env var (Linux)
                          5. DYLD_LIBRARY_PATH env var (osx)
    A library may be 'not found' even if the file exists:
          * library is not compatible (different compiler/bitness)
          * the dependencies are not found
   )
  Tried '/home/m8/miniforge3/envs/var2/lib/python3.10/site-packages/cmeel.prefix/lib/python3.10/site-packages/casadi' :
    Error code: /home/m8/miniforge3/envs/var2/lib/python3.10/site-packages/cmeel.prefix/lib/python3.10/site-packages/casadi/libcasadi_nlpsol_ipopt.so: cannot open shared object file: No such file or directory
  Tried '' :
   

RuntimeError: Error in Opti::value [OptiNode] at .../casadi/core/optistack.cpp:173:
.../casadi/core/optistack_internal.cpp:871: Assertion "solved()" failed:
This action is forbidden since you have not solved the Opti stack yet (with calling 'solve').

In [ ]:
ren.set_state(Q_INIT)

In [ ]:

sleep(5)
for i in range(ns+1):
    ren.set_state(q_opt[:, i])

    for j in range(i):
        ren.markers[j](position=q_opt[:3, j],
                       color=[0, 0, 1, 0.5],
                       size=j*0.02/ns + 0.01)

    ren.markers[ns+1](position=Q_FINAL[:3],
                      color=[1, 0, 0, 0.5],
                      size=0.04)
    ren.markers[ns+2](position=Q_INIT[:3],
                    color=[0, 1, 0, 0.5],
                    size=0.04)
    sleep(t_final/ns)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(q_opt[:3,:].T)
plt.show()

plt.plot(q_opt[3:7,:].T)
plt.show()


In [ ]:
plt.plot(v_opt[:3,:].T)
plt.show()

plt.plot(v_opt[3:,:].T)
plt.show()


In [ ]:

plt.plot(u_opt.T)
plt.show()

In [ ]:
np.linalg.norm(q_opt[3:7,:].T, axis = 1)